In [1]:
import requests
import json
import time
import math
import re
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import os, sys, glob
import kaleido
from PIL import Image
from fpdf import FPDF
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [3]:
def viz(data):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False,
                      title_font_color="blue",
                      title_font_size = 20)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])   
    fig.show()

In [4]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'auto') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
#                           dataset.iloc[indices.flatten()[i]]['O-H'],dataset.iloc[indices.flatten()[i]]['O-L'],dataset.iloc[indices.flatten()[i]]['O-C'],dataset.iloc[indices.flatten()[i]]['H-L'],dataset.iloc[indices.flatten()[i]]['H-C'],dataset.iloc[indices.flatten()[i]]['L-C'],
                           dataset.iloc[indices.flatten()[i]]['col_1'],dataset.iloc[indices.flatten()[i]]['col_2'],dataset.iloc[indices.flatten()[i]]['col_3'],
#                           dataset.iloc[indices.flatten()[i]]['col_4'],dataset.iloc[indices.flatten()[i]]['col_5'],
                           dataset.iloc[indices.flatten()[i]]['F_SMA_10'],dataset.iloc[indices.flatten()[i]]['F_SMA_20'],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
#                      'O-H','O-L','O-C','H-L','H-C','L-C',
                      'col_1','col_2','col_3',
#                      'col_4','col_5',
                      'F_SMA_10','F_SMA_20'
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [5]:
filename = 'EUR_USD_H4.csv'
data = pd.read_csv(filename)

In [6]:
Test_Candle = [ 662, 5870, 4153,  158, 4013, 2017, 5246, 6528,  239, 9372, 3384,
               2147, 6234, 4903, 3691,  724, 3678, 2771, 5729, 6224, 2317, 4696,
               836, 2086, 5121, 5487,  187, 3049, 1921, 5376, 3282, 3046, 7014,
               8611, 9150, 3307,   21, 3411, 6772, 2271, 7612, 3106, 8562, 3869,
               2647, 4374, 6116, 3445, 3709,  812, 2989,  557, 1028, 5038, 7850,
               1018, 8413,  368, 1051, 4725, 9012,  693, 7004, 5533, 4092, 6901,
               1204, 4994, 7893, 4740, 6809, 9516, 6021, 6319,  887, 5830, 6982,
               4984,  138, 8270, 8707, 8401, 9196, 1463, 3526, 1991, 6745, 2980,
               9541, 3643, 5376, 1882, 7955, 1359, 2976, 4819, 6239,  111, 1892,
               1736, 6229, 3681, 7526, 6081, 2825, 6187, 3046, 7027, 7782, 4362,
               7847, 1534, 1574, 1267, 5732,   45, 6293, 2880, 7764, 3989, 6535,
               3199, 9396,  538, 7135, 2873,  353, 3595, 8781, 2130, 5732, 4301,
               1015, 9117, 4531,   29, 1800, 5676, 9309, 8865, 1918,  417, 1987,
               3056, 7418,  124, 8580, 8412,    9, 8311, 3314, 1921, 4201, 4246,
               5705, 6443, 2755, 3975, 2813, 7498, 5165, 5194, 2215, 5274, 6839,
               4597, 8088, 6537, 1269, 8686, 2968, 4410, 2282,  659,  828, 7506,
               6781, 5592, 9591, 4196, 3606, 4494, 1092, 3531, 1746, 3647, 5620,
               3403, 7359, 9273, 8954, 5361, 1736, 8984,  820,  666, 7861,  203,
               4940, 8328, 2971, 8347, 4588, 6826, 6635,  130, 1014, 2651, 4050,
               3858, 6397, 2303, 3767, 4413, 1839, 8711,   83, 5377, 5741, 7730,
               5345, 1647, 9353, 1216, 6479, 2116, 2456, 7593,  428, 7084,  199,
               2147, 6606, 2069, 4689, 1380, 7008, 3253, 4545, 1211, 8396,  730,
               1107, 8345, 3162, 4977,  695,  197, 8293, 6266, 3528,   30, 7985,
               4194, 6971, 7218, 7105, 6035, 6180, 8064, 6491, 7069, 6264, 3142,
                188, 1992, 6515, 1713, 4601, 7422, 5809,  735, 7833, 1431, 7554,
                233, 6874, 8046, 5752, 2604, 5438, 8648, 1491, 7572, 2234, 6047,
               1587, 1440, 7071, 7788, 7177, 9356, 5156, 7942, 8276, 8911,   95,
               1979, 9199, 1752, 2115, 5355, 4167, 9285, 5131, 1444, 2302, 9221,
               1850, 7206, 3115,  139, 2820, 9081, 6478, 5361,  963, 1332, 1085,
               8759, 2753, 6622, 8809, 2928,  566, 8954,  324, 4786,  352, 3057,
               5004, 5273, 4231,   92, 7448, 9088, 3415,  441, 5298, 1649, 2245,
               5756, 3033, 7027, 5598,  586,  364, 4972, 4562, 9196, 7467, 9082,
               7052, 2930, 7997, 7518, 1775, 2054, 8493,  475, 7735, 5481, 6915,
                402, 2443, 3138, 7167, 4478, 4994, 2574, 2285, 7348, 2301, 5370,
                 13, 4736, 7838, 1417, 8399, 1999, 5206, 2360, 6493, 2042,  379,
               2858, 3681, 3617, 2799, 5444, 7616, 2006, 3395, 7984, 6587, 8562,
               3697,  106, 1754, 6468, 2907, 9137, 1575, 4693, 4310, 4405, 7688,
                989, 5317, 1543, 1472, 2288, 9382, 3069, 2849, 2227, 4634, 6095,
                 88, 3959, 6659, 8229, 1083, 4452, 1369, 1023, 1283, 6225, 5183,
               6452,  219, 9063, 7304, 4522,  598, 7567, 3662, 2243, 7589, 9532,
               1024, 4399, 4916, 5462, 7337, 7898, 3635, 7184, 7370, 4224, 1000,
                274,  207, 3827, 9229, 1708, 9313, 9050, 1266,  306, 4111, 9596,
               1598, 6445, 1899, 3633, 6342, 1269, 5107, 1012, 9099, 1569, 8110,
                731, 3629, 6814, 6442, 8639, 2153, 5578, 3917, 3242, 8206,  239,
               7677, 7236, 7064, 1306,  163, 1724, 7744, 5069, 9021, 8776, 9235,
               4363, 3243, 8578, 1076, 6492,  323, 2179,  490,  301, 4762, 1357,
                378,  131,  419, 3111, 1950, 2770, 5022, 7605, 2459,  573, 2221,
               7601, 7128, 7180, 4615, 6705, 8231, 1664, 5451, 2786, 5868, 6842,
               8038, 7855, 4471, 3243, 6708, 1627, 7439, 5483, 5587, 3770, 4937,
               1476, 7357, 5516, 6562, 3391,  634, 2799, 6061, 3493, 2493, 2793,
               7125, 7406, 4579, 5520, 8251, 1458, 3508, 3701, 2335, 4778, 6149,
               3934, 4351, 4616, 4720, 7235, 2833, 1796, 5051, 3610, 7762, 9245,
               4707, 6579, 1761, 8826, 1566, 6898,  663,  310, 4784, 1510, 7235,
               7880, 8040, 3691, 5079, 9406, 6372, 9405, 7214, 4785, 2784, 3332,
               2833, 4916,  269, 2411,  132, 9056, 7058, 7549, 8525,  461, 4297,
               6030, 3198, 1869, 5379,  856, 4758, 6302, 4659, 3624, 7398, 7557,
               6677, 2304, 7527,  733, 9243, 8419, 8357, 4083, 3530, 4493, 4564,
               3865, 1255, 8770, 2827, 5822, 9431, 9600, 2843, 1565, 2480, 6142,
               8545,   23,  301, 3311, 3180, 3200, 8263, 4501, 7335,  948, 7600,
               1319, 8403, 4018, 7412, 3064, 7470, 8564, 8906, 1242, 6164, 3773,
               7309,  351, 2729, 9188, 4875,  273, 8325, 1896, 5214, 5092, 5236,
               5889, 2168, 6936, 5011, 4047, 3780, 1520, 8859, 2099, 6445, 5402,
               4387, 5751, 2937, 1278, 3033, 2929, 2639, 1304, 7849, 6543, 8822,
               7452, 9249,  568, 1727, 7171, 6513, 2802, 9001, 4674, 8117, 9589,
               2612, 4028,  733, 4264,  734, 4630, 9254, 4097,   90, 7351, 4204,
               5105,  683,  696, 2833, 6185, 1341, 9197, 3855, 3863, 2457, 4158,
               3149, 5115, 1856, 1720, 8471, 2033,  820, 5085, 7191, 8835, 9531,
               9533, 1486, 7030, 1482, 1951, 7478, 5834, 5371, 1326,  599, 8689,
               4273,  146, 5921, 3654, 2370, 4234, 8711, 7829, 2502, 3107, 6081,
               3537, 3696, 1109, 7902, 9123, 7287, 9383, 1680, 4186, 8409, 7696,
               4045, 7371, 4531, 1027, 1920, 7666,  915, 9023, 6243, 8781, 4534,
               6463, 8770, 2154, 9290, 4662, 1860, 8493, 9550, 8668, 8196, 8434,
               8215, 4600, 9197, 2050, 6314, 6495, 7214, 3712,  480, 7462, 9264,
               3559, 1837, 6449, 4330, 6362, 6556, 9447, 8510, 2429, 3557, 3086,
               1327, 1090,  835, 6999, 6512, 2547, 2644, 3763, 5307,  955, 8513,
               1515, 6538, 2640, 8534, 5247, 9598, 2972, 9065, 4687, 4860, 8471,
               4016,  347,  668, 9596, 8302, 5320, 3302, 5639, 8152, 1939, 2813,
               8270, 5951, 8145, 3441, 3344,  907, 8971, 5287, 3098, 4225, 7131,
               7731, 7004, 6562, 4718, 5008, 3596, 3848, 8413, 8597, 2784, 8401,
                316,  393, 4795,  496, 7487, 9301, 4266, 9559, 8375, 5391,  140,
               4871, 8832, 2100, 3590, 3490, 6262, 1667,   90, 7671, 5048, 4575,
               1773,  437, 2852,  508, 3188, 3234, 9074, 1027, 1520, 6597, 7651,
               5599, 5114, 3234, 2013, 8285, 5986,  534, 1368, 4743, 4993, 5607,
               5869,  128, 2567, 4555, 2689, 2483, 5963, 2876, 6508, 9356, 9518,
               8723, 8247, 2384, 3761, 5019, 3599, 8931,  748, 9551, 7035, 8973,
               8450, 1611, 3846, 9255,  214, 5748, 7429, 9443, 7841,  553, 9243,
                488, 5078,  691, 1715, 1492, 1447, 1254, 6735, 7766, 7980, 6429,
               1088, 3371, 5621, 4613, 7367, 7259, 1511, 2692, 1724, 4206, 1672,
               7201, 3518, 1476,  211, 4896, 2124, 2193, 3587, 2489, 9001, 8826,
               7259, 6569,   27, 1168, 6194, 6496, 1720, 4203, 2580, 8460, 6308,
               2539, 8015, 6321, 8102, 7385,  208, 5508, 6493, 2461, 6507]

In [7]:
data.shape

(9647, 29)

In [8]:
data.head(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,Direction,col_1,col_2,col_3,col_4,col_5,SMA_10,SMA_20,F_SMA_10,F_SMA_20
0,2015-08-19,01:00:00,1,231,2,Wednesday,2015-08-19T01:00:00.000000000Z,1665,1.10380,1.10526,...,1,-0.00109,0.00037,0.00019,-0.00138,0.00052,1.103502,1.105358,0.001388,-0.000468
1,2015-08-19,05:00:00,5,231,2,Wednesday,2015-08-19T05:00:00.000000000Z,4532,1.10492,1.10734,...,1,-0.00046,0.00196,0.00009,-0.00109,-0.00138,1.103942,1.105107,0.001438,0.000273


# <font color='red'>CANDLE LOOP</font>

In [9]:
%%time

result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
                              'Current_Market_Stoploss':[],
                              'Rec1':[],
                              'Rec1_P':[],
                              'Rec2':[],
                              'Rec2_P':[],
                              'Rec3':[],
                              'Rec3_P':[]
                             })

for candle_no in Test_Candle:

    #print('candle number is = ',candle_no)
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+6]
    #print('data is = ', data)
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred).round(2)*100
    #print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market='Bullish'
        Current_Market_Stoploss= 0
    else:
        Current_Market = 'Bearish'
        Current_Market_Stoploss= 0 
    
    data = pd.read_csv(filename)
    data = data.drop(columns=['Volume','Weekday','Date','Time',
                          'Weekday_Name','UTC_Time','Direction',
                          'Open', 'High', 'Low', 'Close',
                          'O-H','O-L','O-C','H-L','H-C','L-C',
                          'SMA_10','SMA_20',
                          'f_time','julian_date','ATR_14',
#                          'col_1','col_2','col_3',
                          'col_4','col_5',
#                          'F_SMA_10','F_SMA_20'
                         ])

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    predicted_output = []
    for indice in indices[1:4]:
             
        Predicted_Market_Fit =0
        Predicted_Trade=''
    
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+12]

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        Y = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        y_pred = linear_regressor.predict(X)

        Predicted_Market_Fit= r2_score(Y, y_pred).round(2)*100
        coeficient = (linear_regressor.coef_)

        if coeficient > 0:
            Predicted_Trade= 'BUY'
        else:
            Predicted_Trade = 'SELL'
        
        predicted_output.append([Predicted_Market_Fit,Predicted_Trade])

        
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
              'Current_Market_Stoploss':Current_Market_Stoploss,
              'Rec1': predicted_output[0][0],
              'Rec1_P': predicted_output[0][1],
              'Rec2': predicted_output[1][0],
              'Rec2_P': predicted_output[1][1],
              'Rec3': predicted_output[2][0],
              'Rec3_P': predicted_output[2][1],
             }
    
    result_output = result_output.append(result, ignore_index = True)

CPU times: user 3min 5s, sys: 31.6 s, total: 3min 37s
Wall time: 3min 37s


In [10]:
result_output.to_csv('Back_Test_Result.csv', header = True, index = False)
result_output = pd.read_csv('Back_Test_Result.csv')

In [11]:
result_output.head(10)

,Candle_No,Current_Market_Fit,Current_Market,Current_Market_Stoploss,Rec1,Rec1_P,Rec2,Rec2_P,Rec3,Rec3_P
0,662.0,33.0,Bearish,0.0,4.0,BUY,3.0,BUY,80.0,BUY
1,5870.0,67.0,Bearish,0.0,75.0,BUY,29.0,SELL,11.0,SELL
2,4153.0,92.0,Bearish,0.0,22.0,BUY,3.0,SELL,0.0,BUY
3,158.0,33.0,Bearish,0.0,0.0,BUY,26.0,BUY,5.0,SELL
4,4013.0,0.0,Bullish,0.0,74.0,BUY,45.0,BUY,69.0,BUY
5,2017.0,86.0,Bullish,0.0,60.0,BUY,17.0,BUY,37.0,BUY
6,5246.0,2.0,Bearish,0.0,33.0,BUY,91.0,SELL,80.0,SELL
7,6528.0,92.0,Bullish,0.0,80.0,SELL,19.0,BUY,67.0,BUY
8,239.0,86.0,Bullish,0.0,82.0,BUY,0.0,BUY,64.0,SELL
9,9372.0,93.0,Bullish,0.0,41.0,BUY,2.0,BUY,89.0,BUY


In [12]:
result_output.tail(10)

,Candle_No,Current_Market_Fit,Current_Market,Current_Market_Stoploss,Rec1,Rec1_P,Rec2,Rec2_P,Rec3,Rec3_P
990,2539.0,99.0,Bearish,0.0,52.0,BUY,9.0,SELL,38.0,SELL
991,8015.0,1.0,Bullish,0.0,53.0,SELL,52.0,BUY,73.0,SELL
992,6321.0,30.0,Bullish,0.0,20.0,BUY,25.0,BUY,5.0,BUY
993,8102.0,3.0,Bullish,0.0,18.0,SELL,57.0,SELL,37.0,BUY
994,7385.0,18.0,Bullish,0.0,38.0,BUY,66.0,SELL,7.0,BUY
995,208.0,40.0,Bearish,0.0,61.0,BUY,27.0,BUY,76.0,BUY
996,5508.0,41.0,Bearish,0.0,0.0,SELL,60.0,BUY,71.0,BUY
997,6493.0,61.0,Bearish,0.0,94.0,BUY,73.0,SELL,45.0,SELL
998,2461.0,51.0,Bullish,0.0,96.0,SELL,9.0,BUY,14.0,BUY
999,6507.0,46.0,Bearish,0.0,40.0,SELL,85.0,BUY,46.0,SELL
